In [ ]:
go thoroughly through the attached research paper. i have taken a dataset where it has only images and labeled 0 for real and 1 for fake (labels.csv). 

preprocessing is already completed and the preprocessed images are stored in the file path(/home/shree_xd/Documents/deepfake_detection/preprocessed) . do not give code for preprocessing. this preprocessed folder contain images stored as (fake_1.jpeg,fake_2.webp,................,real_1.png,real_2.jpeg.........)
give code for 
Set up EfficientNet for feature extraction
Patch Embedding and Vision Transformer
Combine the features and build the final model
model evaluation accuracy scores and roc 

# EfficientNet-B7

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

# Define EfficientNet-B7 model for feature extraction
def build_efficientnet():
    base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(384, 384, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Build the EfficientNet model
efficientnet_model = build_efficientnet()


# Patch Embedding and Vision Transformer

In [16]:
from tensorflow.keras.layers import Input, Reshape, Dense
from vit_keras import vit, utils

# Patch size and embedding dimensions
patch_size = 32
embedding_dim = 1024

# Input layer for patch embedding
input_patch = Input(shape=(patch_size, patch_size, 3))
reshaped_patch = Reshape((patch_size * patch_size, 3))(input_patch)

# Vision Transformer (ViT) for patch embedding
vit_model = vit.vit_b16(
    image_size=patch_size,
    activation='sigmoid',
    pretrained=True,
    include_top=False,
    pretrained_top=False
)(reshaped_patch)

# Final dense layer for embedding
embedding_output = Dense(embedding_dim)(vit_model)


SyntaxError: incomplete input (2767927898.py, line 90)

In [14]:
print(tf.__version__)

2.16.1


# Combine features and build model

In [17]:
from tensorflow.keras.layers import Concatenate, Dropout

# Inputs for real and fake images
input_real = Input(shape=(384, 384, 3))
input_fake = Input(shape=(384, 384, 3))

# Obtain features from EfficientNet
features_real = efficientnet_model(input_real)
features_fake = efficientnet_model(input_fake)

# Concatenate EfficientNet features with ViT embeddings
combined_features_real = Concatenate()([features_real, embedding_output])
combined_features_fake = Concatenate()([features_fake, embedding_output])

# Final classification layers
dense_real = Dense(1, activation='sigmoid')(combined_features_real)
dense_fake = Dense(1, activation='sigmoid')(combined_features_fake)

# Build the model
model = Model(inputs=[input_real, input_fake], outputs=[dense_real, dense_fake])

# Compile the model
model.compile(optimizer=SGD(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 2560), (None, 1024, 1024)]

# Model Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_curve, auc
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Assuming you have paths to your preprocessed images and labels
image_paths = [...]  # List of image file paths
labels = [...]  # List of corresponding labels (0 for real, 1 for fake)

# Load images and preprocess
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(384, 384))
    img = img_to_array(img) / 255.0  # Normalize pixel values
    return img

# Load images and labels
images = np.array([load_and_preprocess_image(path) for path in image_paths])
labels = np.array(labels)

# Assuming you have a train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Train the model
model.fit([X_train, X_train], [y_train, y_train], epochs=40, batch_size=12, validation_split=0.2, callbacks=[EarlyStopping(patience=5)])

# Evaluate on test data
predictions = model.predict([X_test, X_test])
predictions = np.concatenate(predictions, axis=1).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, np.round(predictions))

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)

print(f"Accuracy: {accuracy}")
print(f"AUC: {roc_auc}")

# Plot ROC curve
import matplotlib.pyplot as plt

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
